#Import all libraries

In [1]:
!pip --q install datasets
!pip --q install deep_translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 733.8 kB/s eta 0:00:00


In [2]:
import warnings
warnings.filterwarnings("ignore")
import json
import os
import copy

from deep_translator import GoogleTranslator
from datasets import DatasetDict,Dataset

import pandas as pd
import torch

# Download and explore the original dataset

 [visualqa](https://visualqa.org/)

In [3]:
versionType  ="v2"          #VQA v2.0 dataset
taskType    ='OpenEnded'    #type of annotations in the JSON file (OpenEnded).
dataType    ='mscoco'       #source of the images (mscoco or abstract_v002).
dataSubType = "train2014"   #type of data subtype (e.g. train2014/val2014/test2015 for mscoco, train2015/val2015 for abstract_v002).

questionsName = f"{versionType}_Questions_Train_{dataType}"




## Download

In [4]:
!mkdir original-dataset

In [5]:
%cd /content/original-dataset/

/content/original-dataset


In [6]:
questionsLink = f"https://s3.amazonaws.com/cvmlp/vqa/{dataType}/vqa/{questionsName}.zip"
!wget {questionsLink}
!unzip -q {questionsName+".zip"}
!rm {questionsName+".zip"}

--2024-04-25 17:55:35--  https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Questions_Train_mscoco.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.197.224, 52.217.201.192, 52.216.245.94, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.197.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7239401 (6.9M) [application/zip]
Saving to: ‘v2_Questions_Train_mscoco.zip’

v2_Questions_Train_ 100%[===================>]   6.90M  25.6MB/s    in 0.3s    

2024-04-25 17:55:36 (25.6 MB/s) - ‘v2_Questions_Train_mscoco.zip’ saved [7239401/7239401]





---

## Explore



In [7]:
quesFile     = f"{versionType}_{taskType}_{dataType}_{dataSubType}_questions.json"

In [8]:
with open(quesFile, "r") as f:
    ques_dict = json.load(f)

ques_dict.keys()

dict_keys(['info', 'task_type', 'data_type', 'license', 'data_subtype', 'questions'])

In [9]:
questions_ds = Dataset.from_list(ques_dict['questions'])
questions_ds

Dataset({
    features: ['image_id', 'question', 'question_id'],
    num_rows: 443757
})

In [10]:
questions_ds[0]

{'image_id': 458752,
 'question': 'What is this photo taken looking through?',
 'question_id': 458752000}

In [11]:
len(questions_ds)

443757



---

# Translation

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
%cd /content/
!mkdir /content/drive/MyDrive/questions-translated-dataset

/content
mkdir: cannot create directory ‘/content/drive/MyDrive/questions-translated-dataset’: File exists


In [14]:
%cd /content/drive/MyDrive/questions-translated-dataset/

/content/drive/MyDrive/questions-translated-dataset


In [15]:
translator = GoogleTranslator(source='auto', target='ar')



---
**Questions Translation**


In [16]:
def questions_translation(batch):
  try:
    sample = batch['question']
    batch["question"] = translator.translate_batch(sample)
    print("len batch:",len(sample))

  except:
     batch["question"] = sample
     print("An except occurred")
     return batch

  return batch

In [17]:
torch.set_num_threads(1)

In [18]:
lends = len(questions_ds)
lends

443757

In [19]:
# range(50000,100000)
# range(200000,250000)
# range(350000,400000)

In [20]:
questions_translation_ds = questions_ds.select(range(350000,400000)).map(
            questions_translation,
            batched=True,
            num_proc = 2,
            desc="Questions Translation",
        )

Questions Translation (num_proc=2):   0%|          | 0/50000 [00:00<?, ? examples/s]

len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch:
 1000len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch:1000 
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 
1000len batch: 1000
len batch: 1000


In [21]:
questions_translation_ds

Dataset({
    features: ['image_id', 'question', 'question_id'],
    num_rows: 50000
})

In [22]:
questions_translation_ds[-1]

{'image_id': 117514,
 'question': 'هل هاتان المركبتان تسيران على نفس المسار؟',
 'question_id': 117514000}

---
**Save Questions**

In [23]:
# section = 2
# section = 5
# section = 8

In [24]:
section = 8

In [25]:
ques_dict['questions'] =  questions_translation_ds.to_list()

In [26]:
with open(f'translated_{section}_{quesFile}', 'w') as f:
  json.dump(ques_dict, f)

In [27]:
json.load(open(f'translated_{section}_{quesFile}','r'))

{'info': {'description': 'This is v2.0 of the VQA dataset.',
  'url': 'http://visualqa.org',
  'version': '2.0',
  'year': 2017,
  'contributor': 'VQA Team',
  'date_created': '2017-04-26 17:07:13'},
 'task_type': 'Open-Ended',
 'data_type': 'mscoco',
 'license': {'url': 'http://creativecommons.org/licenses/by/4.0/',
  'name': 'Creative Commons Attribution 4.0 International License'},
 'data_subtype': 'train2014',
 'questions': [{'image_id': 495269,
   'question': 'ما هو الطبق الذي يقدم؟',
   'question_id': 495269016},
  {'image_id': 495269,
   'question': 'ماذا يوجد في الجزء العلوي من الكعكة؟',
   'question_id': 495269017},
  {'image_id': 495269,
   'question': 'هل توجد فاكهة بين طبقات الكيك؟',
   'question_id': 495269018},
  {'image_id': 495269,
   'question': 'هل تم عض هذه الكعكة؟',
   'question_id': 495269019},
  {'image_id': 495269,
   'question': 'كيف سيكون طعم هذا الزينة على الكعكة؟',
   'question_id': 495269020},
  {'image_id': 495269,
   'question': 'أي نوع من الكعكة هذا؟',
  